In [ ]:
import numpy as np
import pandas as pd
import os
import csv

This just provides the path and filename for the multi-line metadata.tsv file to be split.  You'll have to update it for your environment.

In [ ]:
input_dir = "/home/welling/git/hubmap/ingest-pipeline/src/ingest-pipeline/md/florida_10x_md"
in_fname = os.path.join(input_dir, "UFLA_10XscRNAseq_metadata_050520_-_UFLA_10XscRNAseq_metadata_050520.tsv")

Read the records

In [ ]:
recs = []
with open(in_fname, 'r', newline='') as f:
    dialect = csv.Sniffer().sniff(f.read(128))
    f.seek(0)
    reader = csv.DictReader(f, dialect=dialect)
    for row in reader:
        recs.append({k : v for k, v in row.items()})

Check what we've read.  Beware of columns named '' or ' ' - they indicate a trailing tab and thus an empty column in the input file!

In [ ]:
print(reader.fieldnames)

In [ ]:
print(recs[0])

In [ ]:
print(dialect.delimiter)  # should be tab

Tool to find a uuid in a directory path

In [ ]:
def is_uuid(s):
    return s and len(s) == 32 and all([c in '0123456789abcdef' for c in list(s)])

def get_uuid(s):
    words = s.split('/')
    while words:
        if is_uuid(words[0]):
            return words[0]
        else:
            words = words[1:]
    return None

Write the output single-line metadata.tsv files.
*Output is written to the directory in which this notebook is running.*

In [ ]:
for rawrec in recs:
    rec = rawrec.copy()  # don't stomp on original
    #display(rec)
    uuid = get_uuid(rec['data_path'])
    out_fname = '{}-metadata.tsv'.format(uuid)
    print(out_fname)
    rec['metadata_path'] = '.'
    rec['data_path'] = '.'
    with open(out_fname, 'w', newline='') as f:
        writer = csv.DictWriter(f, dialect=dialect, fieldnames=reader.fieldnames)
        writer.writeheader()
        writer.writerow(rec)